In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_addons as tfa

from dataloader.datasets import DataGenerator
from dataloader.utils import get_count_classes, plot_classes, plot_from_one_class
from models.feature_model import load_and_compile_model

SEED = 0

# Load datasets

In [2]:
data_generator = DataGenerator(
    directory="tiny-imagenet-200/train/",
    batch_size=128,
    image_size=(64, 64),
    shuffle=True,
    seed=SEED,
    validation_split=0.2
)

ds_train, ds_val = data_generator.train, data_generator.val

Found 100000 files belonging to 200 classes.
Using 80000 files for training.
Using 20000 files for validation.
Metal device set to: Apple M1


2022-12-21 11:14:01.869984: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-21 11:14:01.870346: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Load model

In [3]:
model = load_and_compile_model()

In [ ]:
# Train the network
history = model.fit(
    ds_train,
    epochs=1)

2022-12-21 11:14:18.715618: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-21 11:14:18.753085: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


609/625 [============================>.] - ETA: 2s - loss: 0.9958

# Save model

In [ ]:
model.save("saved_models/feature_model")

# Evaluate model

In [ ]:
model.predict(ds_val)

In [ ]:
model.evaluate(ds_val)

In [ ]:
ds_one_class = ds_val.unbatch().filter(lambda x, y: y==100)

In [ ]:
model.evaluate(ds_one_class.batch(32))